In [1]:
# webpage extraction with beautifulsoup
import urllib2
from bs4 import BeautifulSoup

#specify the url
manual = 'https://haynes.com/en-us/car-manuals/car-makes'

#Query the website and return the html to the variable 'page'
page = urllib2.urlopen(manual)

In [2]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
print soup.prettify()

In [4]:
makes = soup.find_all("div", class_="item-list")[1]
links_makes = makes.find_all("div", class_ = "views-field views-field-nothing")

In [5]:
list_makes = []
prefix = "https://haynes.com"
for make in links_makes:
    list_makes.append(prefix + make.a["href"])    

In [6]:
#Query the website and return the html to the variable 'page'
list_models = []

for link in list_makes:
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp, "lxml")
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])


In [10]:
def extract_info(link, result):
    import re
    page_1 = urllib2.urlopen(link)
    soup_1 = BeautifulSoup(page_1)
    title = soup_1.find("h1", class_="product-title").get_text()
    price = soup_1.find("span", class_="price-amount commerce-price-savings-formatter-price").get_text()
            
    body = soup_1.find_all("div", class_="field-item even")
    
    Format = body[2].get_text()
    decription = body[4].get_text() + body[5].get_text()+ body[6].get_text() + body[7].get_text()
    description = decription.encode('utf-8')
    
    table = body[8].get_text()
    part_no = soup_1.findAll('div', text = re.compile('Part number'), attrs = {'class' : 'field-item even'})
    dimensions = soup_1.findAll('div', text = re.compile('Dimensions'), attrs = {'class' : 'field-item even'})
    pages = soup_1.findAll('div', text = re.compile('# of pages'), attrs = {'class' : 'field-item even'})
    ISBN13 = soup_1.findAll('div', text = re.compile('ISBN-13'), attrs = {'class' : 'field-item even'})
    ISBN10 = soup_1.findAll('div', text = re.compile('ISBN-10'), attrs = {'class' : 'field-item even'})
    UPC = soup_1.findAll('div', text = re.compile('UPC'), attrs = {'class' : 'field-item even'})
    publish_date = soup_1.findAll('div', text = re.compile('Publication date'), attrs = {'class' : 'field-item even'})
    language = soup_1.findAll('div', text = re.compile('Language'), attrs = {'class' : 'field-item even'})
    
    reviews = soup_1.find("iframe", class_ = "product-reviews-amazon")
    if reviews is not None:
        reviews = reviews["src"]
    
    result.append([title, price, Format,description, table, part_no, dimensions, pages,ISBN13, ISBN10, UPC, publish_date, language, reviews])

In [ ]:
import time
start = time.time()

result = []
for link in list_models:
    # 3 types of links
    # 1-result of a search, go to next level
    if "search?query" in link:
        page_temp = urllib2.urlopen(link)
        soup_temp = BeautifulSoup(page_temp, "lxml")
        model_temp = soup_temp.find_all("div", class_="product-teaser-title")[1]
        for item in model_temp:
            if item is not None:
                print item
                link_temp = prefix + item["href"]
                extract_info(link_temp, result)
         
    # 2-need to go to next level by "more info"
    elif link[-1].isdigit():
        page_temp = urllib2.urlopen(link)
        soup_temp = BeautifulSoup(page_temp, "lxml")
        link_temp = soup_temp.find_all("div", class_="views-field views-field-view-node")
        for item in link_temp:
            link_temp = prefix + item.a["href"]
            extract_info(link_temp, result)
    
    # 3-final page
    else:
        extract_info(link, result)
        
end = time.time()
print(end - start)
'''
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp)
    
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])
'''

In [ ]:
'''
for item in model_temp:
    print item
    if item:
        print item["href"]
'''

In [ ]:
print len(result)

In [ ]:
import json

with open("Manuals.json", "w") as writeJSON:
    json.dump(result, writeJSON)

In [ ]:
import csv

with open("Manuals.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(result.encode("utf-8"))
'''
with open("Manuals.csv", "w") as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow(["title","price", "Format","decription", "table", "part_no", "dimensions", "pages","ISBN13", "ISBN10", "UPC", "publish_date", "language", "reviews"])
    for a in result:
        writer.writerow(a)
'''

In [ ]:
print soup_temp.prettify()